# Final Project -  KickStarter Campaigns

### ITESM - Escuela de Ingeniería y Ciencias
#### Maestría en Ciencias Computacionales
#### Aprendizaje Automático

---

## Carlos E. Hdz Rincón - A01181616

# Introduction

Crowdfunding has been increasingly getting more visibility and credibility in the past few years after having successfully funded and launched products and services.

However, despite being an option for starting projects, the rate of failed projects is still staggering. The reasons are many, but it often reduces to projects overcomitting and under-delivering promises or to projects understimating how much money they will actually need to successfully kickstart a project.

In this project, we'll try to produce a model that predicts the final state of a project in kickstarter given a set of features. In other words this will be treated as a **classification problem**.

We'll use the dataset compiled by the Kaggle user Kemical on https://www.kaggle.com/kemical/kickstarter-projects.

# Imports and tools

In [1]:
%matplotlib inline

from IPython.display import display, HTML

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

from sklearn.model_selection import train_test_split
from sklearn import metrics

from sklearn.naive_bayes import GaussianNB, CategoricalNB, BernoulliNB
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.preprocessing import RobustScaler
from sklearn.neural_network import MLPClassifier

from imblearn.under_sampling import RandomUnderSampler


# Connect to Google Drive
#from google.colab import drive
#drive.mount('/content/drive')

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
def print_marquee(msg: str):
    marquee_width = len(msg) + 4
    print("\n")
    print("*" * marquee_width)
    print(f"* {msg} *")
    print("*" * marquee_width)

def print_metrics(y_true, y_pred):
    print(metrics.confusion_matrix(y_true, y_pred))
    print("Puntos no categorizados correctamente: %d/%d" 
      % ((y_true != y_pred).sum(), y_true.shape[0]))
    
    print("\n")
    print(metrics.classification_report(y_true, y_pred))

def search_and_evaluate_model(param_searcher, X, y, X_tst, y_tst, 
                              columns_to_drop: list=["ID"]):

    print_marquee("Buscando hiperparámetros óptimos...")

    param_searcher.fit(X.drop(columns_to_drop, axis=1),
                       y)
    
    print("Los mejores parámetros son %s con un score de %0.2f"
        % (param_searcher.best_params_, param_searcher.best_score_))
    

    print_marquee("Reporte de búsqueda de hiperparámetros")
    result_report = pd.DataFrame.from_dict(param_searcher.cv_results_)
    display(result_report)

    print_marquee("Matriz de confusión y reporte de predicciones en dataset de prueba:")
    y_pred = param_searcher.predict(X_tst.drop(columns_to_drop, axis=1))
    print_metrics(y_tst, y_pred)

def model_2_pkl(model, filename: str):
    pickle.dump(model, open(filename, 'wb'))

def pkl_2_model(filename: str):
    return pickle.load(open(filename, 'rb'))

# Data

The following data was pulled from over 300,000 Kickstarter projects spanning successful projects, funded but unsucessful projects or just projects that failed from the outset.

We'll try to predict the final state of the project given the properties of its crowdfunding campaign. We have the following independent variables:

## Independent Variables

1. ID (numerical). The unique ID of the project, doesn't seem to provide any statistica information.
1. name (str). The name of the campaign. We might be able to extract some information from it after tokenizing the field.
1. category (categorical). The subcategory to which the project belongs.
1. main_category (categorical). The parent category to which the project belongs.
1. currency (categorical). The currency used for the funds.
1. deadline (datetime). The hard end date of the campaign, if funds haven't been raised by this date, the project may be deemed failed.
1. goal (float). The fund raising goal for the project expressed in *currency*.
1. launched (datetime). The date at which the campaign was launched.
1. pledged (float). Pledged amount for the project in *currency*.
1. backers (int). The number of individual backers or contributors that have pledged some amount of money to the project.
1. country (categorical). The country from which the campaign was launched.
1. usd_pledged (float). The amount pledged to the project converted to USD by Kickstarter's API.
1. usd_pledged_real (float). The amount pledged to the project converted to USD by fixer.io's real time API. This amount takes into account inflation (i.e. gets the current value of the USD)
1. usd_goal_real (float).  The goal amount of the project converted to USD by fixer.io's real time API. This amount takes into account inflation (i.e. gets the current value of the USD)

## Dependent variable
1. state (categorical). Represents the final status of the crowdfunding campaign, it can be one of the following values:

 1. canceled. If the campaign was canceled by the project owners or site admins.
 1. failed. If the campaign failed to meet its goal by the deadline.
 1. live. If the campaign is still ongoing and its final state hasn't been determined yet.
 1. successful. If the campaign successfully raised at least its goal but is yet to deliver a real product or service.
 1. suspended. If the campaign has been temporarily suspended at the request of the project owners or site admins.
 1. undefined. If the state of a campaign cannot be determined with certainty.



In [ ]:
data_root_location = "/content/drive/My Drive/AprendizajeAutomatico/FinalProject/"
#data_root_location = "/workspace/aprauto/FinalProject/"

data_filename = "ks-projects-201801.csv"

data = pd.read_csv(f"{data_root_location}{data_filename}",
                   header=0,
                   dtype={
                       "category": "category",
                       "main_category": "category",
                       "currency": "category",
                       "state": "category",
                       "country": "category"
                   },
                   parse_dates=["deadline", "launched"])

print(f"Data: {data.shape}")
data.head()

## EDA

Before we begin a proper Exploratory Data Analysis, let's begin with doing some brief description of the data: how many columns and rows there are, what are their names, how many records in each *state* category there are, etc.

In [ ]:
print(f"Data Shape: {data.shape}\n")
print(f"Columns: \n{data.columns.values}\n\n")

print("Datatypes:")
data.info()

print("\n\nState Counts:")
print(data['state'].value_counts())


print("\n\nData Description:")
display(data.describe(include="all", datetime_is_numeric=True))

### Data Description

This first round of analysis brings some really good insight. For starters, we can se that the dataset is quite imbalanced, the *undefined, live and suspended* states are very little in comparison to the failed and successful. We can also see that failed projects are more common on Kickstarter.

We can also see that most of the projects use USD as currency, they usually ask for around \$45,000 USD, have around 100 backers, but they only raise just under \$10,000 USD which seems to show that while crowdfunding has been getting more popular in the past few years, people still pledge to a campaign cautiously.

There is missing data in some of the columns, but we'll discuss that in a bit.



## Missing values

In [ ]:
data.isnull().sum()

*usd_pledged* is missing data but we can ignore the whole column as this is redundant and *usd_pledged_real* has better data.

Name has missing very few records with no information, so we could fill it with "N/A" for now.

We can now see that there's no missing data.

In [ ]:
data["name"].fillna("N/A", inplace=True)
data.drop(["usd pledged"], axis=1, inplace=True)

data.isnull().sum()

## Categorical values

Let's inspect what kind of categories we have and how many of each there are.

In [ ]:
def plot_histogram(df: pd.DataFrame, nrows: int, ncols: int):

    fig, ax = plt.subplots(nrows=nrows,
                           ncols=ncols,
                           figsize=(14,14))
    
    for n, col in enumerate(df.columns):
        plt.subplot(nrows, ncols, n+1)
        df[col].plot.hist(ax = plt.gca(),
                          bins=20,
                          legend=True)

In [ ]:
categorical_cols = [
                "category",
                "main_category",
                "currency",
                "country",
                "state"
]

for cat_col in categorical_cols:
    unique_vals = data[cat_col].cat.categories.values
    print(f"{cat_col}({len(unique_vals)}):")
    print(f"{unique_vals}\n")

In [ ]:
fig_count = plt.figure()

for idx, cat_col in enumerate(categorical_cols):
    ax = fig_count.add_subplot(3, 2, idx + 1)

    data[cat_col].value_counts().plot(kind='bar',
                                      ax=ax,
                                      figsize=(14,14),
                                      legend=True)

plt.show()

As mentioned before, we can see that the dependant categorical column represents an imbalanced dataset.

We can also visually confirm that most of the projects are US-based and thus use USD as its currency.

## Data Transformations and Feature Engineering

After removing the *usd pledge* column and filling the 4 missing values in name. The next things we can do is:

### 1 Drop the pledged and goal column

Much like the *usd pledge* column, since we will use *usd_pledged_real* and *usd_pledged_goal* we can also drop the *pledged* and *goal* columns as they are now redundant.

More over, we want all money data to be in the same currency to better compare apples-to-apples.

In [ ]:
data_fixed = data.copy()

data_fixed.drop(["pledged", "goal"], axis=1, inplace=True)

### 2 Change the category columns to discrete numerical values.


In [ ]:
for cat_col in categorical_cols:    
    data_fixed[cat_col] = data_fixed[cat_col].astype('category').cat.codes


data_fixed[categorical_cols].head(5)


### 3 Create a new column *campaign_duration_days* that stores the difference between the deadline - launched and remove both columns.

In [ ]:
data_fixed["campaign_duration_days"] = \
    (data_fixed.deadline - data_fixed.launched).astype('timedelta64[D]')

data_fixed[["launched", "deadline", "campaign_duration_days"]].head(10)

In [ ]:
data_fixed.drop(["launched", "deadline"], axis=1, inplace=True)
data_fixed.head(5)

### 4 Tokenize the *name* column, remove stop words and encode the presence of the top 50 most common tokens.

Let's first see if there are records that share the same exact name and how many records there are.

In [ ]:
cnt = data_fixed.groupby(["name"]).size().reset_index(name='count')
cnt[cnt["count"] > 1].sort_values(by=['count'], ascending=False)

We see that there are various projects with the same name, we can tokenize and encode the presence of the most common tokens.

We'll use a Tfidf Vectorizer to determine those terms that are most relevant in the whole corpus of the name column, to do this, we'll:
1. Strip any accent characters
1. Remove stop words
1. Limit it to the 50 most important tokens
1. A valid token is one that is at least 2 letters and may have additional alphanumerical characters. The tokens will be converted to lowercase
1. These new 50 features for each row will replace the *name* column in the data, we'll prefix these new columns with *name_* so as to remember where they came from.

In [ ]:
vectorizer = TfidfVectorizer(
    strip_accents="unicode",
    stop_words=ENGLISH_STOP_WORDS,
    binary=False,
    max_features=50,
    token_pattern=u'(?ui)\\b\\w*[a-z]{2,}\\w*\\b'
)
word_tfidf_vector = vectorizer.fit_transform(data_fixed["name"])

print(f"Top TF-IDF Tokens to generate {word_tfidf_vector.shape} tokens:")
print(vectorizer.get_feature_names())

In [ ]:
name_features = pd.DataFrame(word_tfidf_vector.toarray(),
                             columns=vectorizer.get_feature_names())
name_features = name_features.add_prefix("name_")
display(name_features.head(5))

In [ ]:
data_fixed_2 = data_fixed.copy()

data_fixed_2 = pd.concat([data_fixed_2, name_features], axis=1)

display(data_fixed_2)

#### Remove the *name* column and see that all data is now numerical

In [ ]:
data_fixed_2.drop("name", axis=1, inplace=True)
data_fixed_2.info()

## Outlier analysis

Let's begin with a histogram frequency count for each of the columns, we want to ignore the ID as it does not provide significant frequency data. We'll take a look at how the categorical data looks like.

We'll also ignore the *name_* columns given how many and how sparse they are.

In [ ]:
def plot_histogram(df: pd.DataFrame, nrows: int, ncols: int, bins: int=20):

    fig, ax = plt.subplots(nrows=nrows,
                           ncols=ncols,
                           figsize=(14,14))
    
    for n, col in enumerate(df.columns):
        plt.subplot(nrows, ncols, n+1)
        df[col].plot.hist(ax = plt.gca(),
                          bins=bins,
                          legend=True)
# We don't want to plot the name_ features nor the ID column
columns_to_ignore = ["ID"]
columns_to_ignore.extend(list(data_fixed_2.filter(regex = 'name_')))
plot_histogram(data_fixed_2.drop(columns_to_ignore, axis=1), 5, 2)

In these histograms we see that there are categories that represent a big count. The data in these categories is unbalanced specially for the currency and country columns.

Let's see if there are many outliers in the data with a box plot. We'll focus strictly on the numerical data in:
1. usd_goal_real
1. usd_pledged_real
1. backers
1. campaign_duration_days

In [ ]:
numerical_columns = ["usd_goal_real",
             'usd_pledged_real',
             "backers",
             "campaign_duration_days"]

data_fixed_2[numerical_columns].describe()

In [ ]:
data_fixed_2[numerical_columns].plot.box(subplots=True,
                                       layout=(2, 2),
                                       figsize=(14,14));

All columns have many outliers and have varying ranges of the data, which might not matter too much in some models like Decision Trees or Random Forest but will on some others.

Let's zoom in and check  how many records with outliers there are with InterQuartile Range (IQR).

Anything inside of the $IQR*1.5$ will be not be considered an outlier, anything over or under the $IQR*3.0$ will be considered an outlier.

In [ ]:
def describe_outliers_with_iqr(df: pd.DataFrame):

    for col in df.columns:
        column_data = df[col]
        iqr = column_data.quantile(0.75) - column_data.quantile(0.25)
        
        upper_limit = column_data.quantile(0.75) + (iqr * 1.5)
        upper_limit_extreme = column_data.quantile(0.75) + (iqr * 3)

        lower_limit = column_data.quantile(0.25) - (iqr * 1.5)
        lower_limit_extreme = column_data.quantile(0.25) - (iqr * 3)

        num_outliers_below_lower = df[df[col] < lower_limit_extreme].shape[0]
        num_outliers_above_upper = df[df[col] > upper_limit_extreme].shape[0]

        print_marquee(f"Outlier data for column {col}")
        print(f"IQR: {iqr}")
        print(f"Lower limit extreme: {lower_limit_extreme}")
        print(f"Lower limit: {lower_limit}")
        print(f"Upper limit: {upper_limit}")
        print(f"Upper limit extreme: {upper_limit_extreme}")

        print(f"Outliers below the lower limit: {num_outliers_below_lower}")
        print(f"Outliers above the upper limit: {num_outliers_above_upper}")


describe_outliers_with_iqr(data_fixed_2[numerical_columns])

Given the big number of outliers, we can't just trim them out of the dataset, instead we will cap the outliers to be the value of the lower and upper limit extremes.

1. usd_goal_real
 * No outliers below the limit
 * Will cap higher outliers to 56000.0 
1. usd_pledged_real
 * No outliers below the limit
 * Will cap higher outliers to 16107.0
1. backers
 * No outliers below the limit
 * Will cap higher outliers to 218
1. campaign_duration_days
 * Will cap lower outliers to 8
 * Will cap higher outliers to 57

In [ ]:
data_fixed_3 = data_fixed_2.copy()


data_fixed_3.loc[data_fixed_3["usd_goal_real"] > 56000, "usd_goal_real"] = 56000
data_fixed_3.loc[data_fixed_3["usd_pledged_real"] > 16107, "usd_pledged_real"] = 16107
data_fixed_3.loc[data_fixed_3["backers"] > 218, "backers"] = 218

data_fixed_3.loc[data_fixed_3["campaign_duration_days"] < 8, "campaign_duration_days"] = 8
data_fixed_3.loc[data_fixed_3["campaign_duration_days"] > 57, "campaign_duration_days"] = 57

In [ ]:
data_fixed_3[numerical_columns].describe()

In [ ]:
data_fixed_3[numerical_columns].plot.box(subplots=True,
                                       layout=(2, 2),
                                       figsize=(14,14));

In the table and the graphs we can now see that all values in numerical columns are not above or below the $3*IQR$ and that most of the data points are inside of the IQR.

## Data Normalization
Let's further process these four numerical columns in the data by using Robust Scaling.

In robust scaling, we scale each feature of the data set by subtracting the median and then dividing by the interquartile range. The interquartile range (IQR) is defined as the difference between the third and the first quartile and represents the central 50% of the data.

This method comes in handy when working with data sets that contain many outliers because it uses statistics that are robust to outliers (median and interquartile range).

In [ ]:
data_fixed_4 = data_fixed_3.copy()

print("Before scaling:")
display(data_fixed_4[numerical_columns].describe())

print("\n\nScaled data:")
data_fixed_4[numerical_columns] = pd.DataFrame(
    RobustScaler().fit_transform(data_fixed_3[numerical_columns]),
    columns=numerical_columns)

display(data_fixed_4[numerical_columns].describe())

We can now see that after treating the outliers and normalizing the data, all columns are roughly inside a similar range and so are the std deviation and means of the data.

It's important to note that after normalization, data may not directly tell us what its value means, i.e: there can't be negative backers or campaign days. But for the model, the normalization helps it to get better results.

## Correlation Matrix

In [ ]:
corr_matrix = data_fixed_4.drop(["ID"], axis=1).corr(method="pearson")
display(corr_matrix)

Let's get the independent variables that strongly correlate to the dependent variable *state*

In [ ]:
display(corr_matrix[np.abs(corr_matrix["state"]) > 0.4]["state"])

We see that the *backers and usd_pledged_real* are the ones that best correlate to state, which makes sense because the ammount of backers and how much money they contribute to a campaign dictates the success or failure of a project in Kickstarter.

It'd be interesting to see if there are columns that don't really correlate (less than 0.1) to the state variable as these might help simplify the model later on.

In [ ]:
display(corr_matrix[np.abs(corr_matrix["state"]) < 0.1]["state"])

It would appear that the presence of certain words in the name column don't really impact the state of a project, if the models are not performing well, we might want to consider removing them from the training so as to reduce the noise.

## Train/Val/Test Split

We'll use a 80/10/10 split for the train/val/test sets.

In [ ]:
def train_val_test_split(X: pd.DataFrame, y: pd.DataFrame,
                         should_print_summary: bool = True):
    # Train/Test split
    X_train, X_test, y_train, y_test = train_test_split(X,
                                                        y,
                                                        train_size=0.8,
                                                        random_state=42)
    # Val/Test split
    X_val, X_test, y_val, y_test = train_test_split(X_test,
                                                    y_test,
                                                    train_size=0.5,
                                                    random_state=42)

    # Ravel the labels to get it in better shape for the training
    y_train = y_train.values.ravel()
    y_val = y_val.values.ravel()
    y_test = y_test.values.ravel()

    if should_print_summary:
        print(f"X_train: {X_train.shape}")
        print(f"X_val: {X_val.shape}")
        print(f"X_test: {X_test.shape}")

        print(f"y_train: {y_train.shape}")
        print(f"y_val: {y_val.shape}")
        print(f"y_test: {y_test.shape}")

    return X_train, X_val, X_test, y_train, y_val, y_test


data_final = data_fixed_4.copy()

dependent_var = "state";

X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(
    data_final.drop(dependent_var, axis=1),
    data_final[[dependent_var]])


In [ ]:
unique, counts = np.unique(y_train, return_counts=True)
print(data["state"].value_counts())
print(dict(zip(unique, counts)))

# Models

Now, we'll take the data and train on it for the following models:

1. MLP Neural Network
1. SVM
1. Random Forest
1. Decision tree
1. Logistic Regression
1. Gaussian Naïve Bayes

We'll use accuracy as the main metric to evaluate the performance, but we'll keep an eye on the precision, recall and F1-Score in order to break ties.

## MLP Neural Network

In [ ]:
hyperparams_mlp = {
    "activation": ["logistic", "tanh", "relu"],
    "solver": ["adam", "sgd"],
    "alpha": [0.0001, 0.0003, 0.009, 0.01, 0.03, 0.1, 0.3, 0],
    "batch_size": ["auto"],
    "learning_rate": ["constant", "invscaling", "adaptive"],
    "learning_rate_init": [0.001, 0.003, 0.009],
    "shuffle": [True, False],
    "nesterovs_momentum": [True, False],
    }

mlp_search = RandomizedSearchCV(MLPClassifier(random_state=42,
                                              early_stopping=True
                                              ),
                                param_distributions=hyperparams_mlp,
                                verbose=3,
                                n_jobs=-1,
                                cv=5,
                                scoring="accuracy",
                                random_state=42,
                                n_iter=100)

search_and_evaluate_model(mlp_search,
                          X_train,
                          y_train,
                          X_val,
                          y_val)

model_2_pkl(mlp_search.best_estimator_,
            f"{data_root_location}mlp_best_model.pkl")



***************************************
* Buscando hiperparámetros óptimos... *
***************************************
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 24.9min finished


Los mejores parámetros son {'solver': 'adam', 'shuffle': True, 'nesterovs_momentum': False, 'learning_rate_init': 0.009, 'learning_rate': 'invscaling', 'batch_size': 'auto', 'alpha': 0, 'activation': 'logistic'} con un score de 0.93


******************************************
* Reporte de búsqueda de hiperparámetros *
******************************************


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_solver,param_shuffle,param_nesterovs_momentum,param_learning_rate_init,param_learning_rate,param_batch_size,...,param_activation,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,88.525095,56.080082,0.387387,0.084221,sgd,False,False,0.009,constant,auto,...,tanh,"{'solver': 'sgd', 'shuffle': False, 'nesterovs...",0.666326,0.819397,0.720645,0.618404,0.823174,0.729589,0.081570,95
1,441.014786,13.455834,0.237609,0.075608,sgd,True,True,0.001,adaptive,auto,...,relu,"{'solver': 'sgd', 'shuffle': True, 'nesterovs_...",0.920922,0.918760,0.919288,0.920723,0.919419,0.919823,0.000848,52
2,178.315003,48.356476,0.456798,0.054532,adam,True,False,0.001,adaptive,auto,...,tanh,"{'solver': 'adam', 'shuffle': True, 'nesterovs...",0.931750,0.931898,0.931189,0.934274,0.932260,0.932274,0.001058,11
3,184.429929,59.006539,0.306244,0.039649,adam,False,True,0.003,adaptive,auto,...,relu,"{'solver': 'adam', 'shuffle': False, 'nesterov...",0.931321,0.932245,0.930413,0.928316,0.929223,0.930304,0.001410,27
4,138.600102,61.728223,0.187143,0.010830,sgd,False,False,0.003,invscaling,auto,...,relu,"{'solver': 'sgd', 'shuffle': False, 'nesterovs...",0.782276,0.782194,0.771779,0.784930,0.775373,0.779311,0.004918,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,92.929492,25.318263,0.171443,0.011116,adam,False,False,0.009,adaptive,auto,...,relu,"{'solver': 'adam', 'shuffle': False, 'nesterov...",0.930941,0.932707,0.926254,0.930346,0.929966,0.930043,0.002115,33
96,90.781639,32.542482,0.373254,0.070742,sgd,False,False,0.009,constant,auto,...,logistic,"{'solver': 'sgd', 'shuffle': False, 'nesterovs...",0.824861,0.757172,0.843611,0.781084,0.803681,0.802082,0.030672,83
97,137.193595,25.015083,0.259948,0.065743,adam,False,False,0.001,constant,auto,...,logistic,"{'solver': 'adam', 'shuffle': False, 'nesterov...",0.903740,0.904747,0.902931,0.900322,0.902501,0.902848,0.001477,64
98,110.546218,23.326557,0.297393,0.051841,sgd,False,True,0.001,constant,auto,...,tanh,"{'solver': 'sgd', 'shuffle': False, 'nesterovs...",0.892302,0.859704,0.892384,0.870777,0.859668,0.874967,0.014754,76




***********************************************************************
* Matriz de confusión y reporte de predicciones en dataset de prueba: *
***********************************************************************
[[ 2383  1510     0    70     1     0]
 [    3 19501     0   234     1    16]
 [    1   193     0    52     0     0]
 [   12    87     3 13221     0    12]
 [    1   166     0    29     0     0]
 [    0     4     0     1     0   365]]
Puntos no categorizados correctamente: 2396/37866


              precision    recall  f1-score   support

           0       0.99      0.60      0.75      3964
           1       0.91      0.99      0.95     19755
           2       0.00      0.00      0.00       246
           3       0.97      0.99      0.98     13335
           4       0.00      0.00      0.00       196
           5       0.93      0.99      0.96       370

    accuracy                           0.94     37866
   macro avg       0.63      0.59      0.61     37866
weighte

## SVM

Training Support Vector Machines can be pretty expensive, specially when faced with big datasets.

To better tackle this, we'll perform an undersampling on the data so as to reduce the more represented classes of *failed, successful and canceled*.

In order to perform an efficient hyperparamenter search for the SVM, we'll undersample first the *failed, successful and canceled* classes to having only 1000 elements.

We'll check what the best parameter for that search were and we'll proceed use these hyperparamentes and train a new SVM while undersampling only the *failed and successful* states so as to give the model more data to learn from.

We'll do two separate searches, one for linear and RBF SVM models and one just for polynomial SVM. The reason for this is that the *degree hyperparameter* has no effect on the configuration for the models other than polynomials. Adding the degree to the search that includes the other types of kernels would be a waste of time and resources.

We'll choose the best model based on the best score obtained by the RandomSearchCV to train a model with more sample for each of the classes.

### Linear, RBF and Sigmoid Searches

In [ ]:
rus = RandomUnderSampler(random_state=42,
                         replacement=True,
                         sampling_strategy={
                             0:1000,
                             1:1000,
                             3:1000
                         })
rus.fit_resample(data_final.drop(["ID", "state"], axis=1),
                 data_final[[dependent_var]])

data_resampled = data_final.loc[rus.sample_indices_]

X_train_resamp, X_val_resamp, X_test_resamp, \
 y_train_resamp, y_val_resamp, y_test_resamp = train_val_test_split(
    data_resampled.drop(dependent_var, axis=1),
    data_resampled[[dependent_var]],
    False)

hyperparams_svc = {
    "C": [0.01, 0.1, 1, 5, 10],
    "kernel": ["linear",
               "rbf",
               "sigmoid"],
    "gamma": ["scale", "auto"],
    }

svc_search = RandomizedSearchCV(SVC(random_state=42),
                                param_distributions=hyperparams_svc,
                                verbose=3,
                                n_jobs=40,
                                cv=2,
                                scoring="accuracy",
                                random_state=42,
                                n_iter=4)

search_and_evaluate_model(svc_search,
                          X_train_resamp,
                          y_train_resamp,
                          X_val_resamp,
                          y_val_resamp)



***************************************
* Buscando hiperparámetros óptimos... *
***************************************
Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=40)]: Using backend LokyBackend with 40 concurrent workers.
[Parallel(n_jobs=40)]: Done   4 out of   8 | elapsed:    4.5s remaining:    4.5s
[Parallel(n_jobs=40)]: Done   8 out of   8 | elapsed: 11.8min finished


Los mejores parámetros son {'kernel': 'linear', 'gamma': 'auto', 'C': 10} con un score de 0.61


******************************************
* Reporte de búsqueda de hiperparámetros *
******************************************


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kernel,param_gamma,param_C,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,659.741726,43.798342,0.626782,0.012878,linear,auto,10,"{'kernel': 'linear', 'gamma': 'auto', 'C': 10}",0.604506,0.612673,0.608589,0.004084,1
1,195.194815,22.967795,0.664235,0.013153,linear,auto,1,"{'kernel': 'linear', 'gamma': 'auto', 'C': 1}",0.600268,0.607095,0.603681,0.003414,2
2,1.840120,0.017105,1.209968,0.041885,sigmoid,auto,5,"{'kernel': 'sigmoid', 'gamma': 'auto', 'C': 5}",0.319875,0.319723,0.319799,0.000076,3
3,1.890783,0.017492,1.184269,0.015459,sigmoid,auto,1,"{'kernel': 'sigmoid', 'gamma': 'auto', 'C': 1}",0.319875,0.319723,0.319799,0.000076,3




***********************************************************************
* Matriz de confusión y reporte de predicciones en dataset de prueba: *
***********************************************************************
[[ 57   0  33   1   3   8]
 [  1   5  88   1   4   3]
 [  0   1 212  26  13  36]
 [  0   1  55  41   6   1]
 [  0   2 116  11  11  30]
 [  0   0   4   0   0 351]]
Puntos no categorizados correctamente: 444/1121


              precision    recall  f1-score   support

           0       0.98      0.56      0.71       102
           1       0.56      0.05      0.09       102
           2       0.42      0.74      0.53       288
           3       0.51      0.39      0.45       104
           4       0.30      0.06      0.11       170
           5       0.82      0.99      0.90       355

    accuracy                           0.60      1121
   macro avg       0.60      0.47      0.46      1121
weighted avg       0.60      0.60      0.55      1121



### Polynomial SVM Search

In [ ]:
rus = RandomUnderSampler(random_state=42,
                         replacement=True,
                         sampling_strategy={
                             0:1000,
                             1:1000,
                             3:1000
                         })
rus.fit_resample(data_final.drop(["ID", "state"], axis=1),
                 data_final[[dependent_var]])

data_resampled = data_final.loc[rus.sample_indices_]

X_train_resamp, X_val_resamp, X_test_resamp, \
 y_train_resamp, y_val_resamp, y_test_resamp = train_val_test_split(
    data_resampled.drop(dependent_var, axis=1),
    data_resampled[[dependent_var]],
    False)

hyperparams_svc = {
    "C": [0.1, 1, 5, 10],
    "degree": [2,3,4],
    }

svc_search = RandomizedSearchCV(SVC(random_state=42,
                                    kernel="poly",
                                    gamma="scale"),
                                param_distributions=hyperparams_svc,
                                verbose=3,
                                n_jobs=8,
                                cv=2,
                                scoring="accuracy",
                                random_state=42,
                                n_iter=4)

search_and_evaluate_model(svc_search,
                          X_train_resamp,
                          y_train_resamp,
                          X_val_resamp,
                          y_val_resamp)



***************************************
* Buscando hiperparámetros óptimos... *
***************************************
Fitting 2 folds for each of 4 candidates, totalling 8 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   8 | elapsed:    3.3s remaining:    9.9s
[Parallel(n_jobs=8)]: Done   5 out of   8 | elapsed:    5.0s remaining:    3.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    7.1s remaining:    0.0s
[Parallel(n_jobs=8)]: Done   8 out of   8 | elapsed:    7.1s finished


Los mejores parámetros son {'degree': 3, 'C': 10} con un score de 0.55


******************************************
* Reporte de búsqueda de hiperparámetros *
******************************************


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_degree,param_C,params,split0_test_score,split1_test_score,mean_test_score,std_test_score,rank_test_score
0,4.206649,0.109407,0.841262,0.006409,3,10,"{'degree': 3, 'C': 10}",0.550747,0.544846,0.547797,0.002951,1
1,2.759338,0.136014,0.905199,0.011773,2,10,"{'degree': 2, 'C': 10}",0.521749,0.514056,0.517903,0.003846,2
2,2.105956,0.034915,1.111125,0.024427,2,0.1,"{'degree': 2, 'C': 0.1}",0.360250,0.362784,0.361517,0.001267,4
3,5.962379,0.266402,0.786319,0.000698,4,5,"{'degree': 4, 'C': 5}",0.478697,0.497323,0.488010,0.009313,3




***********************************************************************
* Matriz de confusión y reporte de predicciones en dataset de prueba: *
***********************************************************************
[[ 23   0  62   0   0  17]
 [  0   0  86   1   0  15]
 [  0   0 215  18   0  55]
 [  0   0  77  22   0   5]
 [  0   0 133   7   0  30]
 [  0   0   0   0   0 355]]
Puntos no categorizados correctamente: 506/1121


              precision    recall  f1-score   support

           0       1.00      0.23      0.37       102
           1       0.00      0.00      0.00       102
           2       0.38      0.75      0.50       288
           3       0.46      0.21      0.29       104
           4       0.00      0.00      0.00       170
           5       0.74      1.00      0.85       355

    accuracy                           0.55      1121
   macro avg       0.43      0.36      0.34      1121
weighted avg       0.47      0.55      0.46      1121



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


### Best SVM Model training

Based on the scores for the non-poly and poly SVMs, we see that the best model is a non-poly model and we can set the hyperparameters to:

* C: 0.1
* kernel: linear
* gamma: scale

We chose the C value to be low in order to keep the training within reasonable time-constraints.

Given that the more samples are given to the SVM training the more it will take to finish, we'll undersample the *successful and failed* classes to 50,000 samples and leave the rest of the classes as theyr are. During experimentations, it was found that giving more than 50,000 made the training unfeasible in terms of time and computational resources.

This will give us a good middle ground between training performance and accuracy of the model.

In [ ]:
rus = RandomUnderSampler(random_state=42,
                         replacement=True,
                         sampling_strategy={
                             #0:50000,
                             1:50000,
                             3:50000
                         })
rus.fit_resample(data_final.drop(["ID", "state"], axis=1),
                 data_final[[dependent_var]])

data_resampled = data_final.loc[rus.sample_indices_]

X_train_resamp, X_val_resamp, X_test_resamp, \
 y_train_resamp, y_val_resamp, y_test_resamp = train_val_test_split(
    data_resampled.drop(dependent_var, axis=1),
    data_resampled[[dependent_var]],
    False)

svc_model = SVC(random_state=42,
                C=0.1,
                kernel="linear",
                gamma="scale")

svc_model.fit(X_train_resamp.drop(["ID"], axis=1),
              y_train_resamp)

print_marquee("Confusion matrix and prediction report on validation dataset:")

y_pred_svc = svc_model.predict(X_val_resamp.drop(["ID"], axis=1))

print_metrics(y_val_resamp, y_pred_svc)

model_2_pkl(svc_model,
            f"{data_root_location}svc_undersampled_best_model.pkl")



*****************************************************************
* Confusion matrix and prediction report on validation dataset: *
*****************************************************************
[[2212 1408    0  187    0    0]
 [   8 4742    0  288    0    4]
 [   2  226    0   62    0    0]
 [   4  314    0 4703    0    2]
 [   0  163    0   42    0    0]
 [   3  161    0    0    0  168]]
Puntos no categorizados correctamente: 2874/14699


              precision    recall  f1-score   support

           0       0.99      0.58      0.73      3807
           1       0.68      0.94      0.79      5042
           2       0.00      0.00      0.00       290
           3       0.89      0.94      0.91      5023
           4       0.00      0.00      0.00       205
           5       0.97      0.51      0.66       332

    accuracy                           0.80     14699
   macro avg       0.59      0.49      0.52     14699
weighted avg       0.81      0.80      0.79     14699



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Random Forest

In [ ]:
hyperparams_rf = {
    "ccp_alpha": [0.001, 0.01, 0.1, 0, 0.5, 1, 5],
    "criterion": ["gini", "entropy"],
    "n_estimators": [100, 250, 500, 750],
    "oob_score": [True, False],
    "bootstrap": [True, False],
    "min_samples_leaf": [2,3,4,5,6],
    "min_samples_split": [2,3,4,5,6],
    "max_features": [None, "sqrt", "log2"]
    }

rf_search = RandomizedSearchCV(RandomForestClassifier(random_state=42),
                                param_distributions=hyperparams_rf,
                                verbose=3,
                                n_jobs=-1,
                                cv=5,
                                scoring="accuracy",
                                random_state=42,
                                n_iter=100)

search_and_evaluate_model(rf_search,
                          X_train,
                          y_train,
                          X_val,
                          y_val)

model_2_pkl(rf_search.best_estimator_,
            f"{data_root_location}rf_best_model.pkl")



***************************************
* Buscando hiperparámetros óptimos... *
***************************************
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed: 22.4min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed: 138.6min finished


Los mejores parámetros son {'oob_score': False, 'n_estimators': 100, 'min_samples_split': 4, 'min_samples_leaf': 4, 'max_features': None, 'criterion': 'entropy', 'ccp_alpha': 0, 'bootstrap': True} con un score de 0.94


******************************************
* Reporte de búsqueda de hiperparámetros *
******************************************


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_oob_score,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_features,param_criterion,...,param_bootstrap,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2844.235199,79.636345,1.506024,0.117847,False,250,6,6,None,gini,...,False,"{'oob_score': False, 'n_estimators': 250, 'min...",0.522546,0.522546,0.522546,0.522539,0.522539,0.522543,0.000004,43
1,920.990199,41.155065,8.961072,0.670711,True,500,4,3,sqrt,entropy,...,True,"{'oob_score': True, 'n_estimators': 500, 'min_...",0.914188,0.913264,0.913891,0.915260,0.913427,0.914006,0.000708,15
2,1007.924377,43.485701,4.771228,0.437878,True,750,5,6,log2,gini,...,True,"{'oob_score': True, 'n_estimators': 750, 'min_...",0.522546,0.522546,0.522546,0.522539,0.522539,0.522543,0.000004,43
3,0.330593,0.069969,0.000000,0.000000,True,500,2,6,log2,entropy,...,False,"{'oob_score': True, 'n_estimators': 500, 'min_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,81
4,0.362765,0.080977,0.000000,0.000000,True,500,2,4,log2,gini,...,False,"{'oob_score': True, 'n_estimators': 500, 'min_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,3277.312386,29.704442,10.967763,0.102346,False,500,2,4,None,entropy,...,False,"{'oob_score': False, 'n_estimators': 500, 'min...",0.912141,0.912174,0.911960,0.913246,0.912041,0.912312,0.000473,19
96,1333.595473,13.897018,7.225159,1.058430,True,750,2,2,sqrt,gini,...,True,"{'oob_score': True, 'n_estimators': 750, 'min_...",0.900125,0.899482,0.902271,0.898688,0.897763,0.899666,0.001524,23
97,0.306287,0.080170,0.000000,0.000000,True,750,3,5,sqrt,gini,...,False,"{'oob_score': True, 'n_estimators': 750, 'min_...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,84
98,547.225938,8.238249,2.862906,0.483062,False,250,6,5,sqrt,gini,...,False,"{'oob_score': False, 'n_estimators': 250, 'min...",0.899366,0.898937,0.899548,0.898836,0.896327,0.898603,0.001168,24




***********************************************************************
* Matriz de confusión y reporte de predicciones en dataset de prueba: *
***********************************************************************
[[ 2391  1505     0    68     0     0]
 [    9 19567     0   172     0     7]
 [    0   195     0    51     0     0]
 [    1    29     0 13294     0    11]
 [    1   166     0    29     0     0]
 [    0     0     0     0     0   370]]
Puntos no categorizados correctamente: 2244/37866


              precision    recall  f1-score   support

           0       1.00      0.60      0.75      3964
           1       0.91      0.99      0.95     19755
           2       0.00      0.00      0.00       246
           3       0.98      1.00      0.99     13335
           4       0.00      0.00      0.00       196
           5       0.95      1.00      0.98       370

    accuracy                           0.94     37866
   macro avg       0.64      0.60      0.61     37866
weighte

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Decision Tree

In [ ]:
hyperparams_dec_tree = {
    "ccp_alpha": [0.001, 0.01, 0.1, 0, 0.5, 1, 5],
    "criterion": ["gini", "entropy"],
    "min_samples_leaf": [2,3,4,5,6],
    "min_samples_split": [2,3,4,5,6],
    "max_features": [None, "sqrt", "log2"]
    }

decision_tree_search = RandomizedSearchCV(DecisionTreeClassifier(random_state=42),
                                                param_distributions=hyperparams_dec_tree,
                                                verbose=3,
                                                n_jobs=-1,
                                                cv=5,
                                                scoring="accuracy",
                                                random_state=42,
                                                n_iter=100)

search_and_evaluate_model(decision_tree_search,
                          X_train,
                          y_train,
                          X_val,
                          y_val)

model_2_pkl(decision_tree_search.best_estimator_,
            f"{data_root_location}decision_tree_best_model.pkl")



***************************************
* Buscando hiperparámetros óptimos... *
***************************************
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done 128 tasks      | elapsed:   36.5s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.7min finished


Los mejores parámetros son {'min_samples_split': 6, 'min_samples_leaf': 4, 'criterion': 'entropy', 'ccp_alpha': 0.001} con un score de 0.94


******************************************
* Reporte de búsqueda de hiperparámetros *
******************************************


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_samples_split,param_min_samples_leaf,param_criterion,param_ccp_alpha,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,10.762658,2.478436,0.175654,0.172274,4,3,gini,0,"{'min_samples_split': 4, 'min_samples_leaf': 3...",0.907256,0.906265,0.907190,0.908591,0.906115,0.907084,0.000886,31
1,11.134115,0.513069,0.047065,0.001228,3,5,entropy,5,"{'min_samples_split': 3, 'min_samples_leaf': 5...",0.522546,0.522546,0.522546,0.522539,0.522539,0.522543,0.000004,60
2,12.607299,1.092429,0.111024,0.080479,2,5,gini,5,"{'min_samples_split': 2, 'min_samples_leaf': 5...",0.522546,0.522546,0.522546,0.522539,0.522539,0.522543,0.000004,60
3,13.668958,0.930849,0.116151,0.081154,6,3,entropy,0.5,"{'min_samples_split': 6, 'min_samples_leaf': 3...",0.522546,0.522546,0.522546,0.522539,0.522539,0.522543,0.000004,60
4,12.668168,1.231083,0.213177,0.160755,2,3,gini,0,"{'min_samples_split': 2, 'min_samples_leaf': 3...",0.907256,0.906265,0.907190,0.908591,0.906115,0.907084,0.000886,31
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,11.059130,0.248326,0.028290,0.001654,3,3,entropy,5,"{'min_samples_split': 3, 'min_samples_leaf': 3...",0.522546,0.522546,0.522546,0.522539,0.522539,0.522543,0.000004,60
96,10.129872,0.364644,0.030944,0.003764,4,6,gini,0.5,"{'min_samples_split': 4, 'min_samples_leaf': 6...",0.522546,0.522546,0.522546,0.522539,0.522539,0.522543,0.000004,60
97,9.694483,0.507111,0.029041,0.003139,6,4,entropy,0.5,"{'min_samples_split': 6, 'min_samples_leaf': 4...",0.522546,0.522546,0.522546,0.522539,0.522539,0.522543,0.000004,60
98,9.428602,0.285762,0.029326,0.002176,2,6,entropy,5,"{'min_samples_split': 2, 'min_samples_leaf': 6...",0.522546,0.522546,0.522546,0.522539,0.522539,0.522543,0.000004,60




***********************************************************************
* Matriz de confusión y reporte de predicciones en dataset de prueba: *
***********************************************************************
[[ 2383  1510     0    68     0     3]
 [    2 19505     0   241     0     7]
 [    0   192     0    54     0     0]
 [    0   102     0 13222     0    11]
 [    1   165     0    30     0     0]
 [    0     0     0     0     0   370]]
Puntos no categorizados correctamente: 2386/37866


              precision    recall  f1-score   support

           0       1.00      0.60      0.75      3964
           1       0.91      0.99      0.95     19755
           2       0.00      0.00      0.00       246
           3       0.97      0.99      0.98     13335
           4       0.00      0.00      0.00       196
           5       0.95      1.00      0.97       370

    accuracy                           0.94     37866
   macro avg       0.64      0.60      0.61     37866
weighte

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Logistic Regression

In [ ]:
hyperparams = {
    "C": [0.001, 0.01, 0.1, 1, 5, 10, 15, 20, 50, 100],
    "penalty": ["none", "l1", "l2"],
    "solver": ["lbfgs", "saga"]
    }

logistic_regression_search = RandomizedSearchCV(LogisticRegression(random_state=42),
                                                param_distributions=hyperparams,
                                                verbose=3,
                                                n_jobs=-1,
                                                cv=5,
                                                scoring="accuracy",
                                                random_state=2,
                                                n_iter=30)


search_and_evaluate_model(logistic_regression_search,
                          X_train,
                          y_train,
                          X_val,
                          y_val)

model_2_pkl(logistic_regression_search.best_estimator_,
            f"{data_root_location}log_regression_best_model.pkl")



***************************************
* Buscando hiperparámetros óptimos... *
***************************************
Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 80 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 out of 150 | elapsed:   40.8s remaining:  1.7min
[Parallel(n_jobs=-1)]: Done  93 out of 150 | elapsed:  2.0min remaining:  1.2min
[Parallel(n_jobs=-1)]: Done 144 out of 150 | elapsed:  2.6min remaining:    6.5s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  2.7min finished


Los mejores parámetros son {'solver': 'saga', 'penalty': 'l2', 'C': 20} con un score de 0.87


******************************************
* Reporte de búsqueda de hiperparámetros *
******************************************


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_sag.py:330: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_solver,param_penalty,param_C,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,36.120222,1.500972,0.056242,0.019947,lbfgs,none,15,"{'solver': 'lbfgs', 'penalty': 'none', 'C': 15}",0.836579,0.830968,0.838131,0.836296,0.831856,0.834766,0.002823,16
1,36.231441,0.403103,0.049872,0.004563,lbfgs,none,0.001,"{'solver': 'lbfgs', 'penalty': 'none', 'C': 0....",0.836579,0.830968,0.838131,0.836296,0.831856,0.834766,0.002823,16
2,122.702350,1.568953,0.041639,0.004491,saga,l2,20,"{'solver': 'saga', 'penalty': 'l2', 'C': 20}",0.869788,0.868584,0.868765,0.870925,0.866848,0.868982,0.001355,1
3,119.849468,3.039925,0.046178,0.004538,saga,none,100,"{'solver': 'saga', 'penalty': 'none', 'C': 100}",0.869788,0.868584,0.868765,0.870925,0.866848,0.868982,0.001355,1
4,119.035241,1.973680,0.046831,0.003592,saga,l2,1,"{'solver': 'saga', 'penalty': 'l2', 'C': 1}",0.869739,0.868584,0.868666,0.870859,0.866799,0.868929,0.001350,11
5,35.426668,1.868490,0.056694,0.018889,lbfgs,l2,20,"{'solver': 'lbfgs', 'penalty': 'l2', 'C': 20}",0.838676,0.831875,0.827221,0.846810,0.831873,0.835291,0.006821,15
6,121.108475,2.032285,0.048033,0.005836,saga,l2,10,"{'solver': 'saga', 'penalty': 'l2', 'C': 10}",0.869788,0.868584,0.868765,0.870925,0.866848,0.868982,0.001355,1
7,35.777829,1.330775,0.051499,0.003703,lbfgs,l2,5,"{'solver': 'lbfgs', 'penalty': 'l2', 'C': 5}",0.833773,0.825752,0.815370,0.830172,0.841132,0.829240,0.008572,23
8,120.693258,3.263991,0.042903,0.004462,saga,none,50,"{'solver': 'saga', 'penalty': 'none', 'C': 50}",0.869788,0.868584,0.868765,0.870925,0.866848,0.868982,0.001355,1
9,0.158578,0.014941,0.000000,0.000000,lbfgs,l1,10,"{'solver': 'lbfgs', 'penalty': 'l1', 'C': 10}",NaN,NaN,NaN,NaN,NaN,NaN,NaN,27




***********************************************************************
* Matriz de confusión y reporte de predicciones en dataset de prueba: *
***********************************************************************
[[ 2359  1473     0   130     0     2]
 [    2 19022     0   729     0     2]
 [    0   188     0    58     0     0]
 [    0  1820     0 11507     0     8]
 [    1   166     0    29     0     0]
 [    0   222     0    41     0   107]]
Puntos no categorizados correctamente: 4871/37866


              precision    recall  f1-score   support

           0       1.00      0.60      0.75      3964
           1       0.83      0.96      0.89     19755
           2       0.00      0.00      0.00       246
           3       0.92      0.86      0.89     13335
           4       0.00      0.00      0.00       196
           5       0.90      0.29      0.44       370

    accuracy                           0.87     37866
   macro avg       0.61      0.45      0.49     37866
weighte

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Naïve Bayes

One of the side-effects of normalizing the data is that some values that were strictly natural numbers before the normalization, may now have negative real values.

Naïve Bayes models, however, do not accept values under 0. Our other models don't really care about that so just for this one model we'll temporarily use the dataset as it was before the normalization and scaling.

In [ ]:
X_train_nb, X_val_nb, X_test_nb, y_train_nb, y_val_nb, y_test_nb = train_val_test_split(
    data_fixed_3.drop(dependent_var, axis=1),
    data_fixed_3[[dependent_var]])


X_train: (302928, 59)
X_val: (37866, 59)
X_test: (37867, 59)
y_train: (302928,)
y_val: (37866,)
y_test: (37867,)


In [ ]:
nb_model = CategoricalNB()
nb_model.fit(X_train_nb.drop(["ID"], axis=1),
             y_train_nb)

print_marquee("Confusion matrix and prediction report on validation dataset:")

y_pred_bayes = nb_model.predict(X_val_nb.drop(["ID"], axis=1))

print_metrics(y_val_nb, y_pred_bayes)

model_2_pkl(nb_model,
            f"{data_root_location}naivebayes_best_model.pkl")



******************************************************************
* Confusion matrics and prediction report on validation dataset: *
******************************************************************
[[ 1613  1868    17   457     9     0]
 [  146 16513    53  3012    24     7]
 [    6   161    11    68     0     0]
 [   18  1277    48 11954    27    11]
 [    5   154     2    35     0     0]
 [    0     0     0     0     0   370]]
Puntos no categorizados correctamente: 7405/37866


              precision    recall  f1-score   support

           0       0.90      0.41      0.56      3964
           1       0.83      0.84      0.83     19755
           2       0.08      0.04      0.06       246
           3       0.77      0.90      0.83     13335
           4       0.00      0.00      0.00       196
           5       0.95      1.00      0.98       370

    accuracy                           0.80     37866
   macro avg       0.59      0.53      0.54     37866
weighted avg       0.8

## Evaluation on Validation Data Set

After running all of the models on the validation dataset, we find the following results on each of the metrics:

&nbsp;|Accuracy|Macro F1|W F1|
:---  |---:    |---:|---:|
MLP NN|0.94|0.61|0.93|
SVM   |0.80|0.52|0.79|
Random Forest|0.94|0.61|0.93|
Decision Tree|0.94|0.61|0.93|
Logistic Regression|0.87|0.49|0.86|
Naïve Bayes|0.80|0.54|0.79|

We can see that the MLP NN, Random Forest and Decision Tree had similar performance, with really good accuracy overall. Their training times and final size of the fitted model differ though, Decision Tree was the fastest and lightest to train, while the Random Forest took the longest and the more space on disk.

These results look very promising, but we must be cautious given that using the accuracy metric may be deceiving.

By taking a look at the confusion matrices for each of these top models we can see that both the Random Forest and Decision Tree didn't even attempted to classify any record as being of *state* 2 and 4 and this is reflected in the Macro F1-Score.

One big reason for this is how imbalanced the original data set is, to recall, this is the distribution of classes as seen during the EDA:

|State|Counts|
|:---|---:|
|failed     |  197719|
|successful |  133956|
|canceled   |   38779|
|undefined  |    3562|
|live       |    2799|
|suspended  |    1846|

We can see that the *undefined, live and suspended* states are underrepresented when compared against *failed and successful*.

A hypothesis for the fact that the models don't even try to classify any record in states 2 and 4 is because they just don't have enough examples to learn from and that the training is skewing towards the most represented classes.

This is the reason why using accuracy as the only metric can be deceiving: a high value might indicate that it is correctly classifying most of the data, because most of the data belongs to only one or two classes.



# Suggested improvements

We'll try to do something regarding this imbalance of classes and retrain the best models found to see if we can get even better results to try on the test dataset.

There are many ways to tackle this problem, and we will first treat the data to level the count of each class. We could oversample the underreprented classes or we could downscale all classes to a random subset that has similar classes for all.

One thing we can notice is that the the *undefined, live and suspended* columns don't really say much about the data. Let's do the following:

1. Undefined. This shows that the data colected for these projects may be unreliable and given the low number of 3562 records, we can safely remove these without affecting the rest of the dataset.
1. Live. Represents projects that have an active campaign and thus their state is bound to change at any time. Given that we want to predict final states only, and the low number of records, we can remove this from the dataset.
1. Suspended. The suspended status means that a project *may* be restarted at a later undetermined date. For our purposes we can reclassify records in this category as cancelled.

## Refactorization of records with some label

Given that we transformed the categorical state types to discrete numbers, let's figure out who's who first by taking a look at the counts for each and then apply the factorization that we just mentioned.

In [ ]:
print(data_final["state"].value_counts())
print(data['state'].value_counts())

1    197719
3    133956
0     38779
5      3562
2      2799
4      1846
Name: state, dtype: int64
failed        197719
successful    133956
canceled       38779
undefined       3562
live            2799
suspended       1846
Name: state, dtype: int64


In [ ]:
def factorize_redundant_states(df: pd.DataFrame):
    df = df.copy()

    # Drop the undefined
    df.drop(df[df['state'] == 5].index, inplace = True) 

    # Drop the live
    df.drop(df[df['state'] == 2].index, inplace = True)

    # Change those who were suspended to canceled
    df.loc[df["state"] == 4, "state"] = 0

    return df

data_final_clean_labels = factorize_redundant_states(data_fixed_4)
print(data_final_clean_labels['state'].value_counts())

1    197719
3    133956
0     40625
Name: state, dtype: int64


## Resplit the dataset into Train/Val/Test sets

This is necessary in order to keep the proportions after removing records from the dataset

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(
    data_final_clean_labels.drop(dependent_var, axis=1),
    data_final_clean_labels[[dependent_var]])


X_train: (297840, 59)
X_val: (37230, 59)
X_test: (37230, 59)
y_train: (297840,)
y_val: (37230,)
y_test: (37230,)


# Retraining of the best model

The RandomForest, DecisionTree and MLP NN had similar results during the first round of training.

For the best models we'll chose the MLP NN and Decision Tree mainly because of their ease of training and using the best hyperparameters found during previous searches.

## MLP NN

In [ ]:
best_mlp_model = MLPClassifier(random_state=42,
                               early_stopping=True,
                               solver='adam',
                               shuffle=True,
                               nesterovs_momentum=False,
                               learning_rate_init= 0.009,
                               learning_rate='invscaling',
                               batch_size='auto',
                               alpha= 0,
                               activation='relu')

print_marquee("Training of MLP NN Model")

best_mlp_model.fit(X_train.drop(["ID"], axis=1),
                   y_train)

best_mlp_score = best_mlp_model.score(X_train.drop(["ID"], axis=1),
                                      y_train)

print(f"Score for model during training: {best_mlp_score}")

print_marquee("Confusion matrix and evaluation on validation set")

y_val_pred = best_mlp_model.predict(X_val.drop(["ID"], axis=1))

print_metrics(y_val, y_val_pred)

model_2_pkl(best_mlp_model,
            f"{data_root_location}final_mlp_best_model.pkl")



****************************
* Training of MLP NN Model *
****************************
Score for model during training: 0.9418781896320172


*****************************************************
* Confusion matrix and evaluation on validation set *
*****************************************************
[[ 2314  1639   119]
 [    1 19421   273]
 [    1   127 13335]]
Puntos no categorizados correctamente: 2160/37230


              precision    recall  f1-score   support

           0       1.00      0.57      0.72      4072
           1       0.92      0.99      0.95     19695
           3       0.97      0.99      0.98     13463

    accuracy                           0.94     37230
   macro avg       0.96      0.85      0.89     37230
weighted avg       0.95      0.94      0.94     37230



## Decision Tree

In [ ]:
best_dt_model = DecisionTreeClassifier(random_state=42,
                                       min_samples_split= 6,
                                       min_samples_leaf= 4,
                                       criterion= 'entropy',
                                       ccp_alpha= 0.001)

print_marquee("Training of Decision Tree Model")

best_dt_model.fit(X_train.drop(["ID"], axis=1),
               y_train)

best_dt_score = best_dt_model.score(X_train.drop(["ID"], axis=1),
                                    y_train)

print(f"Score for model during training: {best_dt_score}")

print(best_dt_model.score(X_train.drop(["ID"], axis=1),
                       y_train))

print_marquee("Confusion matrix and evaluation on validation set")

y_val_pred = best_dt_model.predict(X_val.drop(["ID"], axis=1))
print_metrics(y_val, y_val_pred)

model_2_pkl(best_dt_model,
            f"{data_root_location}final_dt_best_model.pkl")



***********************************
* Training of Decision Tree Model *
***********************************
Score for model during training: 0.9443627450980392
0.9443627450980392


*****************************************************
* Confusion matrix and evaluation on validation set *
*****************************************************
[[ 2314  1645   113]
 [    1 19450   244]
 [    1   102 13360]]
Puntos no categorizados correctamente: 2106/37230


              precision    recall  f1-score   support

           0       1.00      0.57      0.72      4072
           1       0.92      0.99      0.95     19695
           3       0.97      0.99      0.98     13463

    accuracy                           0.94     37230
   macro avg       0.96      0.85      0.89     37230
weighted avg       0.95      0.94      0.94     37230



---

## Conclusions

We can see that both the MLP NN and Decision Tree were able to get really good results across the board. The decision Tree had a slightly better result than the neural network in this case.

We can also see that the F1-Score improved a lot after applying the refactorization of records and that the overall accuracy of the model didn't suffer for it.

# Test Set Results

We'll use both models to test how well they fare generalizing on new data.

## MLP NN

In [ ]:
print_marquee("MLP NN - Confusion matrix and evaluation on test set")

y_test_pred = best_mlp_model.predict(X_test.drop(["ID"], axis=1))

print_metrics(y_test, y_test_pred)



********************************************************
* MLP NN - Confusion matrix and evaluation on test set *
********************************************************
[[ 2316  1674   120]
 [    0 19609   269]
 [    0   137 13105]]
Puntos no categorizados correctamente: 2200/37230


              precision    recall  f1-score   support

           0       1.00      0.56      0.72      4110
           1       0.92      0.99      0.95     19878
           3       0.97      0.99      0.98     13242

    accuracy                           0.94     37230
   macro avg       0.96      0.85      0.88     37230
weighted avg       0.94      0.94      0.94     37230



## Decision Tree

In [ ]:
print_marquee("Decision Tree - Confusion matrix and evaluation on test set")

y_test_pred = best_dt_model.predict(X_test.drop(["ID"], axis=1))
print_metrics(y_test, y_test_pred)



***************************************************************
* Decision Tree - Confusion matrix and evaluation on test set *
***************************************************************
[[ 2316  1683   111]
 [    0 19628   250]
 [    0    88 13154]]
Puntos no categorizados correctamente: 2132/37230


              precision    recall  f1-score   support

           0       1.00      0.56      0.72      4110
           1       0.92      0.99      0.95     19878
           3       0.97      0.99      0.98     13242

    accuracy                           0.94     37230
   macro avg       0.96      0.85      0.89     37230
weighted avg       0.95      0.94      0.94     37230



---
## Conclusions

We can verify that the results on the test set are similar to those obtained using the training and validation sets.

Again, the decision tree has a slight edge over the MLP Neural Network, but this is very small.

This tells us that the models were able to generalize properly to unseen data.

# Evaluation of the live projects

As part of our improvement phase, we decided to remove those records that had an active campaign, that is, those who still didn't have a final state.

We can repurpose those records and try to predict what their final state will be given their current features. This would effectively simulate real-world data.

As a reminder, this is the mapping of states:
* failed = 1
* successful = 3
* canceled = 0
* live = 2

Let's recover those records that had their state set to 2 (live) and remove this state column as we will now predict them.

In [ ]:
live_projects = data_fixed_4[data_fixed_4['state'] == 2].copy()
display(live_projects.head(5))

live_projects.drop(["state"], axis=1, inplace=True)

,ID,category,main_category,currency,state,backers,country,usd_pledged_real,usd_goal_real,campaign_duration_days,name_adventure,name_album,name_american,name_app,name_art,name_book,name_canceled,name_card,name_cards,name_cd,name_children,name_city,name_clothing,name_collection,name_comic,name_dance,name_debut,name_documentary,name_ep,name_feature,name_festival,name_film,name_food,name_free,name_game,name_help,name_home,name_kids,name_life,name_live,name_love,name_magazine,name_make,name_man,name_mobile,name_movie,name_music,name_new,name_novel,name_playing,name_project,name_record,name_series,name_short,name_story,name_studio,name_time,name_tour,name_video,name_world
40,1000149007,95,12,1,2,0.222222,3,-0.037684,-0.229899,4.000000,0.0,0.0,0.0,0.0,0.0,0.719817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.694164,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
183,1000761521,142,6,13,2,-0.148148,22,-0.078211,-0.037037,2.142857,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
458,1002183790,69,6,5,2,-0.018519,9,-0.015180,-0.321527,-1.714286,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
537,1002599057,55,6,13,2,-0.203704,22,-0.154101,-0.259259,4.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
544,1002629894,70,0,8,2,-0.222222,15,-0.155345,-0.403532,-2.000000,0.0,0.0,0.0,0.0,1.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def get_successful_failed_canceled_projects(df: pd.DataFrame,
                                            original_df: pd.DataFrame) -> tuple:
    successful_ids = df[df["state"]==3]["ID"]
    successful_projects = original_df[original_df['ID'].isin(successful_ids)]

    failed_ids = df[df["state"]==1]["ID"]
    failed_projects = original_df[original_df['ID'].isin(failed_ids)]

    canceled_ids = df[df["state"]==0]["ID"]
    canceled_projects = original_df[original_df['ID'].isin(canceled_ids)]

    return successful_projects, failed_projects, canceled_projects

def get_most_common_in_column(df: pd.DataFrame, columns: list) -> pd.DataFrame:
    cnt = df.groupby(columns).size().reset_index(name='count')
    return cnt[cnt["count"] > 1].sort_values(by=['count'], ascending=False)

## MLP NN

In [ ]:
y_live_mlp_predictions = best_mlp_model.predict(live_projects.drop(["ID"], axis=1))

live_projects_mlp_predictions = live_projects.copy()
live_projects_mlp_predictions["state"] = y_live_mlp_predictions
live_projects_mlp_predictions.head()

live_projects_mlp_predictions["state"].value_counts()

1    2297
3     502
Name: state, dtype: int64

In [ ]:
successful_mlp_projects, \
failed_mlp_projects, _ = get_successful_failed_canceled_projects(live_projects_mlp_predictions,
                                        data)

print_marquee("10 Successful projects as predicted by MLP")
display(successful_mlp_projects.head(10))

print_marquee("10 Failed projects as predicted by MLP")
display(failed_mlp_projects.head(10))



**********************************************
* 10 Successful projects as predicted by MLP *
**********************************************


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real
894,1004434782,DIY BOXCAT by Rato Kim - The Vinyl Art Toy Pla...,DIY,Crafts,USD,2018-01-06,2023.0,2017-12-04 17:26:58,3128.0,live,77,US,3128.00,2023.00
990,1005020458,Tiddly Trailer Bar Co.,Drinks,Food,USD,2018-01-02,9500.0,2017-11-03 21:36:22,9500.0,live,36,US,9500.00,9500.00
1603,100822204,Freedom Ride Jacket | Revolutionizing Urban Cy...,Product Design,Design,USD,2018-01-20,20000.0,2017-11-21 15:02:31,55135.0,live,474,US,55135.00,20000.00
2267,1011784198,Through the Eyes of My Mother,Photobooks,Photography,USD,2018-01-05,500.0,2017-12-29 23:15:36,349.0,live,7,US,349.00,500.00
3338,1017252588,"My Debut EP - ""Shine!"" - PHASE 1",Pop,Music,USD,2018-01-09,5000.0,2017-12-10 22:44:19,5416.0,live,41,US,5416.00,5000.00
4039,1020721729,Edensoles | Sole To Soul Comfort with Dynamic ...,Product Design,Design,USD,2018-01-11,5000.0,2017-12-12 18:50:44,53099.0,live,962,US,53099.00,5000.00
4134,1021242831,Plume: the Omnibus,Comics,Comics,USD,2018-01-12,25000.0,2017-12-11 16:00:02,43085.0,live,842,US,43085.00,25000.00
4985,1025450545,"Mazie Meadows Morning Show Podcast, Season 2",Performance Art,Art,USD,2018-01-22,500.0,2017-12-23 00:14:45,500.0,live,1,US,500.00,500.00
5584,1028614956,Journeys; a set of regional pokemon polaroid e...,Accessories,Fashion,CAD,2018-01-03,1900.0,2017-12-04 16:14:01,5550.0,live,121,CA,4436.45,1518.78
5725,1029297597,VO-Viginti Octo Luxury Automatic Moon Phase Watch,Product Design,Design,HKD,2018-01-22,100000.0,2017-12-23 14:12:50,203112.0,live,121,HK,25959.46,12780.86




******************************************
* 10 Failed projects as predicted by MLP *
******************************************


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real
40,1000149007,Unschooling To University Book Project,Nonfiction,Publishing,CAD,2018-01-19,3000.0,2017-11-20 18:15:14,592.0,live,24,CA,472.88,2396.36
183,1000761521,KILOS.,Thrillers,Film & Video,USD,2018-02-06,5000.0,2017-12-23 20:34:31,310.0,live,4,US,310.00,5000.00
458,1002183790,The Man in the Field,Horror,Film & Video,GBP,2018-01-08,850.0,2017-12-21 11:19:18,413.0,live,11,GB,563.32,1159.38
537,1002599057,Dear Atlantas,Film & Video,Film & Video,USD,2018-02-16,2000.0,2017-12-18 02:48:27,5.0,live,1,US,5.00,2000.00
544,1002629894,Art Calendar 2018,Illustration,Art,MXN,2018-01-06,1000.0,2017-12-21 05:34:12,0.0,live,0,MX,0.00,52.32
627,100301082,"Feliz año nuevo, mamá.",Theater,Theater,MXN,2018-02-04,49000.0,2017-12-06 03:36:06,1000.0,live,2,MX,52.32,2563.83
641,1003104071,One GuitarMan Band,Music,Music,EUR,2018-01-17,2000.0,2017-12-18 00:02:46,0.0,live,0,IT,0.00,2427.39
752,1003665581,Durango Falls TV Show,Film & Video,Film & Video,USD,2018-01-20,10000.0,2017-11-21 10:56:11,1.0,live,1,US,1.00,10000.00
783,1003819770,GrandLo Café,Drinks,Food,USD,2018-01-19,25000.0,2017-11-20 19:54:51,9097.0,live,38,US,9097.00,25000.00
842,100411349,E Coin Mining and Rig-Building Workshop,Hardware,Technology,USD,2018-01-06,3100.0,2017-11-22 20:28:28,225.0,live,9,US,225.00,3100.00


In [ ]:
print_marquee("Top Successful Categories - MLP")

display(get_most_common_in_column(successful_mlp_projects,
                                  ["main_category", "category"]).head(10))

print_marquee("Top Failed Categories - MLP")
display(get_most_common_in_column(failed_mlp_projects,
                                  ["main_category", "category"]).head(10))



***********************************
* Top Successful Categories - MLP *
***********************************


,main_category,category,count
749,Design,Product Design,80
1408,Games,Tabletop Games,43
2128,Technology,Gadgets,17
2205,Technology,Technology,15
10,Art,Art,15
1680,Music,Music,13
70,Art,Illustration,12
1378,Games,Playing Cards,11
802,Fashion,Apparel,11
797,Fashion,Accessories,10




*******************************
* Top Failed Categories - MLP *
*******************************


,main_category,category,count
749,Design,Product Design,157
802,Fashion,Apparel,98
2075,Technology,Apps,79
1420,Games,Video Games,78
1680,Music,Music,67
2205,Technology,Technology,65
1408,Games,Tabletop Games,65
993,Film & Video,Documentary,54
2023,Publishing,Publishing,54
1009,Film & Video,Film & Video,51


## Decision Tree

In [ ]:
y_live_dt_predictions = best_dt_model.predict(live_projects.drop(["ID"], axis=1))

live_projects_dt_predictions = live_projects.copy()
live_projects_dt_predictions["state"] = y_live_dt_predictions
live_projects_dt_predictions.head()

live_projects_dt_predictions["state"].value_counts()

1    2283
3     516
Name: state, dtype: int64

In [ ]:
successful_dt_projects, \
failed_dt_projects, _ = get_successful_failed_canceled_projects(live_projects_dt_predictions,
                                        data)

print_marquee("10 Successful projects as predicted by DT")
display(successful_dt_projects.head(10))

print_marquee("10 Failed projects as predicted by DT")
display(failed_dt_projects.head(10))



*********************************************
* 10 Successful projects as predicted by DT *
*********************************************


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real
894,1004434782,DIY BOXCAT by Rato Kim - The Vinyl Art Toy Pla...,DIY,Crafts,USD,2018-01-06,2023.0,2017-12-04 17:26:58,3128.0,live,77,US,3128.00,2023.00
990,1005020458,Tiddly Trailer Bar Co.,Drinks,Food,USD,2018-01-02,9500.0,2017-11-03 21:36:22,9500.0,live,36,US,9500.00,9500.00
1603,100822204,Freedom Ride Jacket | Revolutionizing Urban Cy...,Product Design,Design,USD,2018-01-20,20000.0,2017-11-21 15:02:31,55135.0,live,474,US,55135.00,20000.00
3338,1017252588,"My Debut EP - ""Shine!"" - PHASE 1",Pop,Music,USD,2018-01-09,5000.0,2017-12-10 22:44:19,5416.0,live,41,US,5416.00,5000.00
4039,1020721729,Edensoles | Sole To Soul Comfort with Dynamic ...,Product Design,Design,USD,2018-01-11,5000.0,2017-12-12 18:50:44,53099.0,live,962,US,53099.00,5000.00
4134,1021242831,Plume: the Omnibus,Comics,Comics,USD,2018-01-12,25000.0,2017-12-11 16:00:02,43085.0,live,842,US,43085.00,25000.00
4985,1025450545,"Mazie Meadows Morning Show Podcast, Season 2",Performance Art,Art,USD,2018-01-22,500.0,2017-12-23 00:14:45,500.0,live,1,US,500.00,500.00
5584,1028614956,Journeys; a set of regional pokemon polaroid e...,Accessories,Fashion,CAD,2018-01-03,1900.0,2017-12-04 16:14:01,5550.0,live,121,CA,4436.45,1518.78
5725,1029297597,VO-Viginti Octo Luxury Automatic Moon Phase Watch,Product Design,Design,HKD,2018-01-22,100000.0,2017-12-23 14:12:50,203112.0,live,121,HK,25959.46,12780.86
6418,1032831948,Twisted Goblin Armouries LARP.,Crafts,Crafts,GBP,2018-02-06,1000.0,2017-12-08 15:40:56,1296.0,live,17,GB,1767.71,1363.98




*****************************************
* 10 Failed projects as predicted by DT *
*****************************************


,ID,name,category,main_category,currency,deadline,goal,launched,pledged,state,backers,country,usd_pledged_real,usd_goal_real
40,1000149007,Unschooling To University Book Project,Nonfiction,Publishing,CAD,2018-01-19,3000.0,2017-11-20 18:15:14,592.0,live,24,CA,472.88,2396.36
183,1000761521,KILOS.,Thrillers,Film & Video,USD,2018-02-06,5000.0,2017-12-23 20:34:31,310.0,live,4,US,310.00,5000.00
458,1002183790,The Man in the Field,Horror,Film & Video,GBP,2018-01-08,850.0,2017-12-21 11:19:18,413.0,live,11,GB,563.32,1159.38
537,1002599057,Dear Atlantas,Film & Video,Film & Video,USD,2018-02-16,2000.0,2017-12-18 02:48:27,5.0,live,1,US,5.00,2000.00
544,1002629894,Art Calendar 2018,Illustration,Art,MXN,2018-01-06,1000.0,2017-12-21 05:34:12,0.0,live,0,MX,0.00,52.32
627,100301082,"Feliz año nuevo, mamá.",Theater,Theater,MXN,2018-02-04,49000.0,2017-12-06 03:36:06,1000.0,live,2,MX,52.32,2563.83
641,1003104071,One GuitarMan Band,Music,Music,EUR,2018-01-17,2000.0,2017-12-18 00:02:46,0.0,live,0,IT,0.00,2427.39
752,1003665581,Durango Falls TV Show,Film & Video,Film & Video,USD,2018-01-20,10000.0,2017-11-21 10:56:11,1.0,live,1,US,1.00,10000.00
783,1003819770,GrandLo Café,Drinks,Food,USD,2018-01-19,25000.0,2017-11-20 19:54:51,9097.0,live,38,US,9097.00,25000.00
842,100411349,E Coin Mining and Rig-Building Workshop,Hardware,Technology,USD,2018-01-06,3100.0,2017-11-22 20:28:28,225.0,live,9,US,225.00,3100.00


In [ ]:
print_marquee("Top Successful Categories - DT")

display(get_most_common_in_column(successful_dt_projects,
                                  ["main_category", "category"]).head(10))

print_marquee("Top Failed Categories - DT")
display(get_most_common_in_column(failed_dt_projects,
                                  ["main_category", "category"]).head(10))



**********************************
* Top Successful Categories - DT *
**********************************


,main_category,category,count
749,Design,Product Design,81
1408,Games,Tabletop Games,45
2128,Technology,Gadgets,19
2205,Technology,Technology,17
70,Art,Illustration,15
1680,Music,Music,13
10,Art,Art,13
802,Fashion,Apparel,11
1378,Games,Playing Cards,11
2216,Technology,Wearables,11




******************************
* Top Failed Categories - DT *
******************************


,main_category,category,count
749,Design,Product Design,156
802,Fashion,Apparel,98
2075,Technology,Apps,79
1420,Games,Video Games,78
1680,Music,Music,67
2205,Technology,Technology,63
1408,Games,Tabletop Games,63
993,Film & Video,Documentary,54
2023,Publishing,Publishing,54
1009,Film & Video,Film & Video,51


---

## Conclusions

We can see that both models predicted that most of the projects would fail. The model doesn't expect any of the projects in the *live* category to be canceled.

Interestingly, both models kept relatively the same failed/successful ratio which again confirms their similarity as seen during the validation and test phases.

|State|MLP|Decision Tree|
|:---|---:|---:|
|Failed|2297|2283|
|Successful|502|516|





# Final Conclusions

As with any Data Science project, EDA and data treatment is essential to a good model. Data is everything, no matter how great the hyperparamentes for the model are, the model will output garbage if it is fed garbage.

In this particular case we had to:

1. Choose to use the USD variant of the columns in order to be able to compare apples to apples and dropped the original pledged and goal columns
1. Do feature engineering and create a new column called *campaign_duration_days* out of the deadline and launched columns.
1. Use the name column and apply tokenization to get the top 50 most common tokens which proved to be useful on models such as the MLP Neural Network and decision trees.
1. Do a proper treatment of outliers using the Interquartile Range (IQR)
1. Do data scaling and normalization so that numerical columns had similar values and didn't skew the model too much.

We also noticed that the *backers* and *usd_pledge_real* data correlate better to the dependen variable *state* and this was confirmed in the validation, test and live datasets. We can see that, in general, if a project has less backers or is further away from the goal, the highest the probability that the project will fail.

Naive Bayes, SVM had the worst performance out of all models and logistic regression didn't do too bad.

We got the best results with MLP Neural Networks, Decision Tree and Random Forest. We had 94% accuracy in these models, however, we had less-than ideal F1-Scores which takes into account the precision and recall of the model. This is the reason why using accuracy as the only metric can be misleading, we needed to pay attention to why we were getting such low F1-Scores.

We found that the undefined, suspended and live projects were underrepresented and that they were hardly classified by any model. But we decided that undefined projects didn't have relevant data so we could remove them from training; the suspended projects could arguably be grouped with the canceled projects. And most importantly, the live projects could be used as real-world data to use our best model with.

After factorizing these columns, we could confirm that the models still had a 94% accuracy but a much better F1-Score of 0.89-0.94 for macro and weighted scores. Which tells us that the model is more confident about the kind of predictions it is making.

The best models were Decision Tree and MLP Neural Networks, chosen both because of their performance on the validation set, the relative ease of configuration and less amount of training time.

It's worth noting that while Decision Trees work really good on this kind of problems, they have trouble performing well extrapolating to data that was not seen during training, columns that contain time series such as dates are a good example of this.

MLP Neural Networks, on the other hand, are better at extrapolating data that was not seen during training, which for this particular problem is probable that will happen with projects that have periods of time that are bigger than the ones seen during training.

Given these reasons, MLP Neural Network would probably be a better fit to continuosly make predictions on new data from crowdfunding projects.

Finally, the data shows that Product design, tabletop games and gadgets are the type of projects that have a better chance to succeed on Kickstarter. Paradoxically, product design is also one of the categories that usually fail along with fashion apparel and apps. The fact that product design appears in both categories might be due to it being the biggest category in the platform.

The amount of days of a campaign, the number of backers and how far the pledged money is from the goal seem to be better factors at predicting the final state of a campaing, which confirms what was seen in the correlation matrix.